In [2]:
bucket = "sagemaker-us-east-1-160951647621"
data_path = "data-augmentation-2020-10-05-00-11-55-811/output/classifier/train"

inputs = {
    "train": f"s3://{bucket}/{data_path}"
}

In [23]:
train_instance_type = "ml.p3.2xlarge"
hyperparameters = {
    #"max-rows": 1024,
    "num-epochs": 10,
    "batch-size": 1024,
}

In [26]:
sagemaker.__version__

'2.9.2.dev0'

In [27]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='src/train_classif.py',
                       base_job_name='train-classifier',
                       instance_type=train_instance_type,
                       instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(), # Passes to the container the AWS role that you are using on this notebook
                       framework_version='2.1.0',
                       py_version='py3')

In [28]:
estimator.fit(inputs)

2020-10-05 01:03:41 Starting - Starting the training job...
2020-10-05 01:03:43 Starting - Launching requested ML instances......
2020-10-05 01:05:08 Starting - Preparing the instances for training......
2020-10-05 01:06:06 Downloading - Downloading input data...
2020-10-05 01:06:18 Training - Downloading the training image.........
2020-10-05 01:07:56 Training - Training image download completed. Training in progress.2020-10-05 01:08:00,903 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-10-05 01:08:01,357 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "batch-size": 1024,
        "num-epochs": 10,
        "model_dir": "s3://sag